### **AI project**

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# import sys
# sys.path.append('/content/drive/MyDrive/AI')

In [ ]:
# from erfnet import ERFNet

In [ ]:
!pip install ood_metrics

In [ ]:
# import erfnet
# !git clone https://github.com/shyam671/AnomalySegmentation_CourseProjectBaseCode.git

Cloning into 'AnomalySegmentation_CourseProjectBaseCode'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 61 (delta 3), reused 3 (delta 3), pack-reused 51
Receiving objects: 100% (61/61), 21.48 MiB | 31.74 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [ ]:
# !git clone https://github.com/SegmentMeIfYouCan/road-anomaly-benchmark.git

Cloning into 'road-anomaly-benchmark'...
remote: Enumerating objects: 868, done.
remote: Counting objects: 100% (868/868), done.
remote: Compressing objects: 100% (388/388), done.
remote: Total 868 (delta 541), reused 802 (delta 477), pack-reused 0
Receiving objects: 100% (868/868), 2.72 MiB | 8.17 MiB/s, done.
Resolving deltas: 100% (541/541), done.


In [ ]:
# !git clone https://github.com/gpleiss/temperature_scaling.git

In [ ]:
! pip install wget
# !wget https://github.com/shyam671/AnomalySegmentation_CourseProjectBaseCode/raw/main/trained_models/erfnet_pretrained.pth




  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=11bd718f4b6332de8359f394411ce1b769a1a25af8a43093d7c40b786cb363d9
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
import os
import sys
shared_folder_path = '/content/drive/MyDrive/temperature_scaling'
files_in_shared_folder = os.listdir(shared_folder_path)
sys.path.append('/content/drive/MyDrive/temperature_scaling')

In [ ]:
import sys
import wget

sys.path.append('/content/drive/MyDrive/AnomalySegmentation_CourseProjectBaseCode/eval')
sys.path.append('/content/drive/MyDrive/road-anomaly-benchmark-master/methods')
sys.path.append('/content/drive/MyDrive/temperature_scaling/methods')

import temperature_scaling
from temperature_scaling import ModelWithTemperature
import baselines
from iouEval import *
import eval_iou
from baselines import *
import os
import cv2
import glob
import torch
import random
from PIL import Image
import numpy as np
import erfnet
from erfnet import ERFNet
import os.path as osp
from argparse import ArgumentParser
from ood_metrics import fpr_at_95_tpr, calc_metrics, plot_roc, plot_pr, plot_barcode
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve, average_precision_score

seed = 42

# general reproducibility
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

NUM_CHANNELS = 3
NUM_CLASSES = 20
# gpu training specific
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True


def main():

    anomaly_score_list = []
    ood_gts_list = []
    means_iou = []
    iou_per_class = []

    input = "/content/drive/MyDrive/Dataset/Validation_Dataset/Validation_Dataset/RoadAnomaly/images/*.jpg"
    input_labels_masks = "/content/drive/MyDrive/Dataset/Validation_Dataset/Validation_Dataset/RoadAnomaly/labels_masks/*.png"
    image_files = glob.glob(input)
    label_files = glob.glob(input_labels_masks)
    loadDir = '/content/drive/MyDrive/AnomalySegmentation_CourseProjectBaseCode/trained_models'
    loadWeights = '/content/drive/MyDrive/AnomalySegmentation_CourseProjectBaseCode/trained_models/erfnet_pretrained.pth'
    loadModel = '/content/drive/MyDrive/AnomalySegmentation_CourseProjectBaseCode/eval/erfnet.py'
    subset = 'val'
    batch_size = 1
    cpu = False

    if not os.path.exists('results.txt'):
        open('results.txt', 'w').close()
    file = open('results.txt', 'a')

    modelpath = loadModel
    weightspath = loadWeights

    # print("Loading model: " + modelpath)
    # print("Loading weights: " + weightspath)

    model = ERFNet(NUM_CLASSES)

    if (not cpu):
        model = torch.nn.DataParallel(model).cuda()

    def load_my_state_dict(model, state_dict):  # custom function to load model when not all dict elements
        own_state = model.state_dict()
        for name, param in state_dict.items():
            if name not in own_state:
                if name.startswith("module."):
                    own_state[name.split("module.")[-1]].copy_(param)
                else:
                    # print(name, " not loaded")
                    continue
            else:
                own_state[name].copy_(param)
        return model

    model = load_my_state_dict(model, torch.load(weightspath, map_location=lambda storage, loc: storage))
    print("Model and weights LOADED successfully")
    model.eval()

    temperatures = [0, 0.5, 0.75, 1.1]
    for tem in temperatures:
        new_model = ModelWithTemperature(model, tem)

        msp = Max_softmax(model)
        iou = iouEval(NUM_CLASSES)

        # print = (len(label_files), len(image_files))

        for idx, path in enumerate(image_files):
            # print(path)
            label_path = label_files[idx]
            images = torch.from_numpy(np.array(Image.open(path).convert('RGB'))).unsqueeze(0).float()
            images = images.permute(0, 3, 1, 2)
            label_mask = torch.from_numpy(np.array(Image.open(label_path))).unsqueeze(0).float()
            label_mask = label_mask.unsqueeze(0)
            label_mask = label_mask.permute(0, 3, 1, 2)
            with torch.no_grad():
                result = model(images)
            anomaly_result = msp.anomaly_score(images)  # Max_Softmax_Probability(MSP) score analysis function


            pathGT = path.replace("images", "labels_masks")
            if "RoadObsticle21" in pathGT:
                pathGT = pathGT.replace("webp", "png")
            if "fs_static" in pathGT:
                pathGT = pathGT.replace("jpg", "png")
            if "RoadAnomaly" in pathGT:
                pathGT = pathGT.replace("jpg", "png")

            mask = Image.open(pathGT)
            ood_gts = np.array(mask)

            if "RoadAnomaly" in pathGT:
                ood_gts = np.where((ood_gts == 2), 1, ood_gts)
            if "LostAndFound" in pathGT:
                ood_gts = np.where((ood_gts == 0), 255, ood_gts)
                ood_gts = np.where((ood_gts == 1), 0, ood_gts)
                ood_gts = np.where((ood_gts > 1) & (ood_gts < 201), 1, ood_gts)

            if "Streethazard" in pathGT:
                ood_gts = np.where((ood_gts == 14), 255, ood_gts)
                ood_gts = np.where((ood_gts < 20), 0, ood_gts)
                ood_gts = np.where((ood_gts == 255), 1, ood_gts)
            ood_gts_ = torch.Tensor(ood_gts).unsqueeze(0).unsqueeze(0)
            anomaly_result_ = torch.Tensor(anomaly_result).unsqueeze(0).unsqueeze(0)

            iou.reset()   #correct
            # print(anomaly_result_.shape,ood_gts_.shape)
            iou.addBatch(anomaly_result_, ood_gts_)  #correct
            # miou, iou_arr = iou.getIoU
            f_iou = iou.getIoU()  # correct

            # print(f'mean_IoU in each iteration: {f_iou[0].item()}')
            # print(f'IoU_per_class in each iteration: {f_iou[1][0].item()}')

            means_iou.append(f_iou[0].item())
            iou_per_class.append(f_iou[1][0].item())

            if 1 not in np.unique(ood_gts):
                continue
            else:
                ood_gts_list.append(ood_gts)
                anomaly_score_list.append(anomaly_result)
            del result, anomaly_result, ood_gts, mask
            torch.cuda.empty_cache()

        final_iou_mean = sum(means_iou) / len(means_iou)
        # file.write("\n")

        ood_gts = np.array(ood_gts_list)
        anomaly_scores = np.array(anomaly_score_list)

        ood_mask = (ood_gts == 1)
        ind_mask = (ood_gts == 0)

        ood_out = anomaly_scores[ood_mask]
        ind_out = anomaly_scores[ind_mask]

        ood_label = np.ones(len(ood_out))
        ind_label = np.zeros(len(ind_out))

        val_out = np.concatenate((ind_out, ood_out))
        val_label = np.concatenate((ind_label, ood_label))

        prc_auc = average_precision_score(val_label, val_out)
        fpr = fpr_at_95_tpr(val_out, val_label)

        print(f'AUPRC score for temperature = {tem}: {prc_auc * 100.0}')
        print(f'FPR@TPR95 for temperature = {tem}: {fpr * 100.0}')

        # file.write(('    AUPRC score:' + str(prc_auc * 100.0) + '   FPR@TPR95:' + str(fpr * 100.0)))
        # file.close()

        print(f'final mean_IoU for temparture = {tem}: {final_iou_mean}')


main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model and weights LOADED successfully
AUPRC score for temperature = 0: 9.714782598829546
FPR@TPR95 for temperature = 0: 95.09486059822227
final mean_IoU for temparture = 0: 0.047447599712171044
AUPRC score for temperature = 0.5: 9.714782598829546
FPR@TPR95 for temperature = 0.5: 95.09486059822227
final mean_IoU for temparture = 0.5: 0.04744759971217106
AUPRC score for temperature = 0.75: 9.714782598829546
FPR@TPR95 for temperature = 0.75: 95.09486059822227
final mean_IoU for temparture = 0.75: 0.04744759971217106


In [ ]:
# label_mask.shape

In [ ]:
# print(label_mask.shape)

In [ ]:
# !ls

drive  sample_data


In [ ]:
# !python /content/AnomalySegmentation_CourseProjectBaseCode/eval/evalAnomaly.py --input '/content/drive/MyDrive/AI/Validation_Dataset/FS_LostFound_full/images/*.png' --loadDir ./

In [ ]:
# !git clone https://github.com/SegmentMeIfYouCan/road-anomaly-benchmark.git

In [ ]:
# !wget https://github.com/SegmentMeIfYouCan/road-anomaly-benchmark/raw/master/methods/baselines.py

In [ ]:
#!python /content/road-anomaly-benchmark/methods/baselines.py --input '/content/drive/MyDrive/AI/Validation_Dataset/FS_LostFound_full/images/*.png' --loadDir ./

In [ ]:
# !python /content/AnomalySegmentation_CourseProjectBaseCode/eval/evalAnomaly.py --input '/content/drive/MyDrive/AI/Validation_Dataset/FS_LostFound_full/images/*.png' --loadDir ./

In [ ]:
# !wget /content/drive/MyDrive/AI/prediction_to_cocoformat.py

In [ ]:
# !wget https://github.com/SegmentMeIfYouCan/road-anomaly-benchmark/raw/master/methods/utils.py

In [ ]:
# !pip install utils
#from prediction_to_cocoformat import Max_softmax

In [ ]:
#import utils
#from utils import get_softmax

In [ ]:
# !wget https://github.com/SegmentMeIfYouCan/road-anomaly-benchmark/raw/master/methods/utils.py

In [ ]:
# import utils

In [ ]:
# !pip install wget

In [ ]:
# !pip intall .utils

In [ ]:
# !ls

In [ ]:
# from baselines import Max_softmax

In [ ]:
# A = Max_softmax("RYiJpr3NGZ3PWER")